<img src="https://i.imgur.com/d28OeSq.png" width="350px" align="right">

# Finance with Python Video 2 

Credit to Dr. Yves Hilpisch of The Python Quants GmBH for instructional material

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Option Replication and Pricing

In [2]:
'''Stocks'''
S = np.array((20., 5.)) 

In [3]:
'''S at time zero'''
S0 = 10.

In [4]:
'''Riskless bond asset'''
B = np.array((11.,11.))

In [5]:
'''Bond at time zero'''
B0 = 10.

In [6]:
'''Market matrix'''
M = np.array((S, B)).T
M

array([[20., 11.],
       [ 5., 11.]])

In [7]:
'''Market price vector'''
M0 = np.array((S0,B0)).T
M0

array([10., 10.])

In [8]:
0.5 * S - 0.5 * B

array([ 4.5, -3. ])

In [9]:
K = 15. # strike price of call option
C = np.maximum (S - K, 0) # payoff of call option
C

array([5., 0.])

In [10]:
'''Solve for the optimal portfolio'''
'''shows we should long .33 bond and short the bond .1515 this 
is not replicable in the real world because of partial shares isssues'''
phi = np.linalg.solve(M, C) # solving for M(market) & C(Call Option)
phi #replication portfolio

array([ 0.33333333, -0.15151515])

In [11]:
np.dot(M, phi)# portfolio payoff = option payoff

array([5.00000000e+00, 2.22044605e-16])

In [12]:
C0 = np.dot(M0, phi)# cost to set up portfolio
C0

1.8181818181818188

1.818 is the arbitrage free price of the option 

## Fundamental Theorem of Asset Pricing

Is a martingale measure and the absence of arbitrage are equivalent
http://pi.math.cornell.edu/~mec/Summer2008/spulido/fftap.html

### Probability Measure and Expectation 

In [13]:
'''explicit modeling represented as 2d matrix; up & down where P == Up state'''
P = 0.5
P = np.array((P, 1 - P))
P

array([0.5, 0.5])

In [14]:
'''expected value of S'''
ES = np.dot(S, P)
ES

12.5

In [15]:
'''expected value of B'''
EB = np.dot(B,P)
EB

11.0

### Maritingale Property 

In [16]:
'''interest rate'''
i = 0.1

df = 1 / (1 + i)
df

0.9090909090909091

$Q$ is called a Martingale measure if 

> $\frac{1}{1+i}\cdot E^Q(S_1) = S_0$    

holds

In [17]:
df * np.dot(S, P)

11.363636363636363

In [18]:
S0 # we are looking for S0, the above did not give 10.0
# under P, the price process was not a price process

10.0

## Deriving a Martingale Measure 

In [19]:
'''minimize an error measure under a constraint'''
from scipy.optimize import minimize

In [20]:
'''derive an error measure'''
def error (Q):
    return (np.dot(S, Q) / (1+i) - S0) ** 2 # add the square for convex optimizers; a differentiable smooth function

In [21]:
'''call error on P, the up|down 2d matrix'''
error(P)

1.859504132231404

In [22]:
'''constraint: the sum of the two vectors need to add to 1'''
'''type equation function: lambda Q: Q is supposed to == 1'''
cons = ({'type': 'eq', 'fun': lambda Q: Q.sum() - 1}) #this is a probability constraint

In [23]:
'''boundaries: 2 elements over which we are going to optimize & the prob measure shall be between 0 and 1'''
bnds = ((0,1),(0,1))

In [24]:
''''''
res = minimize(error,(0.5, 0.5), bounds=bnds, constraints=cons)
res

     fun: 1.0172028584007344e-14
     jac: array([ 1.25849558e-06, -6.09001821e-07])
 message: 'Optimization terminated successfully.'
    nfev: 9
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([0.39999999, 0.60000001])

In [25]:
round(res['fun'], 5)

0.0

In [26]:
# x is from array shown above
Q = res['x']
Q

array([0.39999999, 0.60000001])

In [27]:
error(Q)

1.0172028584007344e-14

In [28]:
np.dot(S,Q) / (1 + i)

9.999999899143525

In [29]:
S0

10.0

### Risk Neutral ('Martingale') Pricing 

In [30]:
phi

array([ 0.33333333, -0.15151515])

In [31]:
np.dot(M, phi)

array([5.00000000e+00, 2.22044605e-16])

In [32]:
np.dot(M0, phi)

1.8181818181818188

In [33]:
np.dot(C, P) / (1 + i) # wrong measure OR wrong discount rate

2.2727272727272725

In [34]:
np.dot(C, Q) / (1 + i) # martingale meas in combo with risk free rate

1.8181817845629937

### Aproximation by Ordinary Least Squares

In [35]:
'''same as phi & np.linalg.solve'''
reg = np.linalg.lstsq(M,C)[0]
reg

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  


array([ 0.33333333, -0.15151515])

In [36]:
np.dot(M, reg)

array([5.00000000e+00, 6.66133815e-16])

In [37]:
np.dot(M0, reg)

1.8181818181818188

### Option Pricing Based on Neural Nets ("Learning')

In [38]:
M # layer 0

array([[20., 11.],
       [ 5., 11.]])

In [39]:
'''guessing the portfolio by replication with neural nets'''
'''initial weights vector'''
w = np.array((1., -1.))

In [40]:
'''layer 1'''
layer1 = np.dot(M, w)  
layer1

array([ 9., -6.])

In [41]:
'''deltas'''
d = layer1 - C
d

array([ 4., -6.])

In [42]:
'''mean squared error'''
(d ** 2).mean()

26.0

In [43]:
'''set alpha to [x]'''
alpha = 0.01

In [44]:
'''update the weight vector - the "learning" step '''
w -= alpha * d # with in place substraction
w

array([ 0.96, -0.94])

In [53]:
for _ in range(150):  # range needs to be 150 for teaching purposes :: matches arb free price of ~1.818 at 150
    layer1 = np.dot(M, w)
    d = layer1 - C
    print((d ** 2).mean())
    w -= alpha * d

1.7729916163179096e-09
1.539226254576981e-09
1.3362823833480757e-09
1.1600962514542644e-09
1.0071399050191329e-09
8.743505437482512e-10
7.590691914284266e-10
6.589874524578808e-10
5.721012885211035e-10
4.966708897255833e-10
4.3118583658436344e-10
3.743348553575931e-10
3.2497956112844093e-10
2.82131662727227e-10
2.449331731247443e-10
2.1263922920553212e-10
1.846031765369193e-10
1.6026362075137816e-10
1.3913318621385145e-10
1.2078875677177544e-10
1.0486300327367831e-10
9.10370282019665e-11
7.903398000040985e-11
6.861350944784897e-11
5.956695687901023e-11
5.171317399210625e-11
4.489489650451783e-11
3.897559512257364e-11
3.383674166471219e-11
2.937543565144982e-11
2.5502343819866392e-11
2.2139911323354693e-11
1.922080875835719e-11
1.6686583964651873e-11
1.4486491589872088e-11
1.2576476946209528e-11
1.0918293877037618e-11
9.478738891679353e-12
8.228986323320916e-12
7.144011105069731e-12
6.2020876780907056e-12
5.384354950632078e-12
4.674438632104332e-12
4.058123343671113e-12
3.52306798137082

In [54]:
w

array([ 0.33333333, -0.15151515])

In [55]:
layer1 = np.dot(M, w)
layer1.round(5)

array([ 5., -0.])

In [56]:
np.dot(M0, w)

1.818181817907517

## What was going on above: 

<img src="https://i.imgur.com/z96a78Z.png" width="720px" align="center">

<font color=red size=1> <div align="right">__Image credits to Dr. Yves Hilpisch and The Python Quants__

<img src="https://i.imgur.com/gbscHEI.png" width="720px" align="center">

<font color=red size=1> <div align="right">__Image credits to Dr. Yves Hilpisch and The Python Quants__

## Mean-Variance Portfolio Theory 

In [85]:
'''Array of 3 Payoffs '''
S = np.array((20, 10, 5.))
'''Array of 3 Riskless Payoffs '''
B = np.array((11., 11, 11))
'''Call Option'''
C = np.maximum(S - K, 0)
'''Market'''
M = np.array((S, B, C)).T
M

array([[20., 11.,  5.],
       [10., 11.,  0.],
       [ 5., 11.,  0.]])

In [86]:
'''all rows and first 2 columns'''
M[:, :2]

array([[20., 11.],
       [10., 11.],
       [ 5., 11.]])

In [90]:
reg = np.linalg.lstsq(M[:,:2], C, rcond=None)[0] # had to pass rcond to stop update msg
reg

array([ 0.35714286, -0.22727273])

In [92]:
'''3 payoff state least squares regression'''
np.dot(M[:,:2], reg)

array([ 4.64285714,  1.07142857, -0.71428571])

In [97]:
'''another numerical check with exact solver replicating zero vector'''
np.linalg.solve(M, np.array((0, 0, 0)))

array([ 0.,  0., -0.])

In [98]:
'''another numerical check with exact solver replicating another vector for complete market'''
np.linalg.solve(M, np.array((1, 2, 3)))

array([-0.2       ,  0.36363636,  0.2       ])

## Returns 

In [103]:
'''option payoff is non-redudant cannot use replication arguments IOT come up with unique arb free price'''
'''3-sum vector'''
C0 = 1.25 # <--- Option Payoff
M0 = np.array((S0, B0, C0))
M0

array([10.  , 10.  ,  1.25])

In [111]:
'''Market Matrix'''
M

array([[20., 11.,  5.],
       [10., 11.,  0.],
       [ 5., 11.,  0.]])

In [113]:
'''Return Matrix'''
R = M / M0 -1
R

array([[ 1. ,  0.1,  3. ],
       [ 0. ,  0.1, -1. ],
       [-0.5,  0.1, -1. ]])

In [114]:
'''setting a probability vector'''
P = np.ones(3) / 3
P

array([0.33333333, 0.33333333, 0.33333333])

In [116]:
'''assign R.mean to a vector variable'''
r = R.mean(axis=0) # stock, bonds, option

## Portfolio Return

In [117]:
'''portfolio weights assumed to add up to 1'''
phi = np.array((0.3, 0.4, 0.3))

In [124]:
'''the portfolio return'''
np.dot(r, phi)

0.19

## Volatilities 

Generally defined as the standard deviation of the returns

In [125]:
'''R is the returns matrix'''
R

array([[ 1. ,  0.1,  3. ],
       [ 0. ,  0.1, -1. ],
       [-0.5,  0.1, -1. ]])

In [127]:
R.std(axis=0) # stocks, bonds, option :: simple model structure and rather extreme S & C values

array([0.62360956, 0.        , 1.88561808])

<img src="https://i.imgur.com/d28OeSq.png" width="350px" align="right">